In [ ]:
!pip install implicit scikit-learn pandas openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import os

# SUBIR ARCHIVOS
from google.colab import files

print("Por favor, sube los archivos: Excel con freelancers, texto del cliente y stopwords.")
uploaded = files.upload()

# Cargar el archivo Excel
excel_filename = [f for f in uploaded.keys() if f.endswith('.xlsx')][0]
df_freelancers = pd.read_excel(excel_filename)

# Cargar archivo de texto con la respuesta del cliente
txt_filename = [f for f in uploaded.keys() if f.endswith('.txt') and "RespuestaCliente" in f][0]
with open(txt_filename, "r", encoding="utf-8") as file:
    lines = file.readlines()

# Cargar archivo de stopwords
stopwords_filename = [f for f in uploaded.keys() if f.endswith('.txt') and "stopwords" in f][0]
with open(stopwords_filename, "r", encoding="utf-8") as file:
    stopwords_list = file.read().splitlines()

# Verificar la carga de datos
print("Archivos cargados correctamente.")
print(df_freelancers.head())
print("Primera línea del archivo de texto:", lines[0])
print("Ejemplo de stopwords:", stopwords_list[:10])

Por favor, sube los archivos: Excel con freelancers, texto del cliente y stopwords.


Saving Freelancers_Limpieza_Sintetico_Realista.xlsx to Freelancers_Limpieza_Sintetico_Realista.xlsx
Saving RespuestaCliente1.txt to RespuestaCliente1.txt
Saving stopwords.txt to stopwords.txt
Archivos cargados correctamente.
   PreferedName                                RespuestaFormulario  \
0  Freelancer_1  2,1,1,3,1,1,2,2,2,3,3,3,1,2,3,2,2,3,2,3,2,2,1,2,3   
1  Freelancer_2  1,2,1,3,2,2,2,3,2,2,1,1,2,2,1,1,2,2,2,1,1,3,3,1,3   
2  Freelancer_3  5,5,4,4,5,4,4,5,4,5,5,5,4,5,5,4,4,5,5,5,4,5,4,5,4   
3  Freelancer_4  4,5,3,5,3,5,4,4,3,3,3,3,5,3,3,3,5,5,3,5,3,5,3,3,3   
4  Freelancer_5  3,4,3,4,4,2,3,4,2,3,2,3,3,4,3,2,3,3,4,3,4,3,3,3,3   

                                              Skills  
0  Ad Copy, Blog Writing, Copywriting, Editor, Pr...  
1                                                NaN  
2  Art Design, Branding, Logo, HTML, Print, UX, G...  
3  Blog Writing, Creative Writing, Long Format, S...  
4  Node, Docker, React, Javascript, CSS, HTML, We...  
Primera línea del archiv

In [ ]:
# Renombrar columnas para consistencia
df_freelancers.rename(columns={'PreferedName': 'Freelancer',
                               'RespuestaFormulario': 'Formulario',
                               'Skills': 'Habilidades'}, inplace=True)

# Convertir respuestas del formulario a listas numéricas
df_freelancers['Formulario'] = df_freelancers['Formulario'].apply(
    lambda x: np.array(list(map(int, x.split(',')))) if isinstance(x, str) else np.zeros(25))

# Convertir habilidades en listas de palabras clave
df_freelancers['Habilidades'] = df_freelancers['Habilidades'].apply(
    lambda x: x.lower().split(', ') if isinstance(x, str) else [])

# Extraer respuestas del formulario del cliente (primera línea)
cliente_formulario = np.array(list(map(int, lines[0].strip().split(','))))

# Extraer texto del cliente (resto del archivo)
cliente_texto = " ".join(lines[1:]).strip()

# Vectorizar habilidades usando TF-IDF con stopwords personalizadas
vectorizer = TfidfVectorizer(stop_words=stopwords_list)
all_skills = [" ".join(skills) for skills in df_freelancers['Habilidades']]
skills_matrix = vectorizer.fit_transform(all_skills)

# Vectorizar el texto del cliente con stopwords eliminadas
cliente_texto_vectorizado = vectorizer.transform([cliente_texto])

# Normalizar los vectores del formulario (freelancers y cliente)
scaler = StandardScaler()
formulario_matrix = np.vstack(df_freelancers['Formulario'].values)
formulario_matrix = scaler.fit_transform(formulario_matrix)
cliente_formulario = scaler.transform(cliente_formulario.reshape(1, -1))

# Unificar espacio vectorial (concatenar respuestas del formulario + habilidades)
freelancer_vectors = np.hstack((formulario_matrix, skills_matrix.toarray()))
cliente_vector = np.hstack((cliente_formulario, cliente_texto_vectorizado.toarray()))

# Convertir a matriz dispersa para Implicit
freelancer_vectors_sparse = csr_matrix(freelancer_vectors)
cliente_vector_sparse = csr_matrix(cliente_vector)

print("Transformación de datos completada. Lista para Implicit.")

Transformación de datos completada. Lista para Implicit.


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['able', 'ableabout', 'about', 'above', 'abroad', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'ad', 'added', 'adj', 'adoptar', 'adopted', 'ae', 'af', 'afecta', 'afectado', 'afectando', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'ag', 'again', 'against', 'ago', 'ah', 'ahead', 'ai', 'ain', 'aint', 'al', 'algo', 'algunas', 'algunos', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'announce', 'another', 'ante', 'antes', 'anunciar', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'ao', 'aparecer', 'apart', 'apparently', 'appear', 'appreciate', 'appropriate', 'appro

In [ ]:
import scipy.sparse as sparse

# Crear una matriz dispersa (usuario-item) con los datos de los freelancers
freelancer_matrix = sparse.csr_matrix(freelancer_vectors)

# Convertir el vector del cliente a una matriz dispersa
cliente_matrix = sparse.csr_matrix(cliente_vector.reshape(1, -1))

print("Matrices dispersas creadas correctamente para Implicit.")


Matrices dispersas creadas correctamente para Implicit.


In [ ]:
import implicit

# Convertir la matriz freelancer_vectors a formato disperso para Implicit
freelancer_matrix = sparse.csr_matrix(freelancer_vectors)

# Configurar y entrenar el modelo ALS
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=20)
model.fit(freelancer_matrix)

print("Modelo ALS entrenado con éxito.")



  0%|          | 0/20 [00:00<?, ?it/s]

Modelo ALS entrenado con éxito.


In [ ]:
# Implicit espera que los usuarios sean las filas, así que debemos asegurarnos de que pasamos un usuario válido
user_id = 0  # El cliente es el único usuario en este caso

# Obtener recomendaciones para el cliente
recommended_freelancers = model.recommend(user_id, freelancer_matrix[user_id], N=5)

# Mostrar los freelancers recomendados
recommended_freelancers_df = df_freelancers.iloc[[rec[0] for rec in recommended_freelancers]][['Freelancer', 'Habilidades']]

print("Top Freelancers Recomendados según Implicit ALS:")
print(recommended_freelancers_df)


Top Freelancers Recomendados según Implicit ALS:
       Freelancer                                        Habilidades
54  Freelancer_55  [art design, graphic, logo, print, animation, ...
0    Freelancer_1  [ad copy, blog writing, copywriting, editor, p...


In [ ]:
# Obtener las palabras más relevantes del vectorizador
feature_names = vectorizer.get_feature_names_out()

# Extraer los pesos de las habilidades
skills_weights = skills_matrix.toarray().mean(axis=0)

# Crear un DataFrame con las palabras más importantes
df_keywords = pd.DataFrame({
    'Palabra': feature_names,
    'Peso': skills_weights
}).sort_values(by='Peso', ascending=False)

# Mostrar las palabras más relevantes en la recomendación
print("Palabras clave más relevantes en la vectorización de habilidades:")
print(df_keywords.head(20))


Palabras clave más relevantes en la vectorización de habilidades:
        Palabra      Peso
9           art  0.183199
85      writing  0.124309
13     branding  0.097648
30      graphic  0.086491
21       design  0.083850
23    direction  0.082297
40         logo  0.076392
66       social  0.075960
43        media  0.075960
49    photoshop  0.073062
25      editing  0.073062
52        print  0.067991
77           ux  0.057627
69   storyboard  0.053624
76         user  0.052217
34    interface  0.049867
84   wireframes  0.048902
32         html  0.046304
26       editor  0.045767
17  copywriting  0.044508
